In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import math as math
from scipy import stats
import matplotlib.pyplot as plt

In [32]:
ps4=pd.read_excel("tweets_ps4.xlsx")
ps4=ps4.loc[:,"Treinamento"]
for i in ps4:
#     tweets=[]
    a=i.split()
    for j in a:       
        if j[0] == "@":
            del(j)
#         elif j[:2] == "rt":
#             a.remove(j)
#         elif j[:2] == "rt":
#             a.remove(j)
    print(a)

['@luminescentruby', 'nice.', "i'm", 'an', 'avid', 'ps4', 'player', 'if', 'you', 'ever', 'need', 'a', 'punch', 'bag', 'xd', 'i', 'want', 'a', 'switch', 'but', 'i', "can't", 'justify', 'buying', 'one', 'just', 'for', 'smash', 'and', 'botw']
['hey', 'all', 'ps4', 'users...am', 'i', 'the', 'only', 'one', 'counting', 'down', 'the', 'minutes', 'till', 'spider-man', 'comes', 'out', 'tonight?']
['gucci', 'gamer', '-', 'red', 'knight', '8', 'kill', 'duo', 'win', '#fortnite', '#ps4', '#gamer', 'https://t.co/hfelikihtm']
['rt', '@bravointel:', 'the', 'term', "'dlc'", 'is', 'censored', 'in', 'battlefield', "v's", 'pc', 'in-game', 'chat.', 'https://t.co/xn0hqycbcg']
['rt', '@tegtournaments:', 'fortnite', '2v2', '(ps4)', 'time:', '9:00pm', 'est', 'prize:', '$20', 'join:', 'https://t.co/sgr6fqu6vq', '@dnrrts', '@famertr', '@fear_rts', '@findm…']
['rt', '@uchikoshi_eng:', 'legendary', 'visual', 'novel', '428:', 'shibuya', 'scramble', 'is', 'out', 'on', 'ps4', 'and', 'pc!', 'i', 'joined', 'chunsoft', 